In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from fraud import collectAllFeaturesBaseline


df_fraud_aggregated = collectAllFeaturesBaseline() 

/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/fraud.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  self._frames[str(p)] = pd.read_csv(p)


           client_id  target  disrict  region  client_catg
0     train_Client_0     0.0       60     101           11
1     train_Client_1     0.0       69     107           11
2    train_Client_10     0.0       62     301           11
3   train_Client_100     0.0       69     105           11
4  train_Client_1000     0.0       62     303           11


In [5]:
total = len(df_fraud_aggregated)
report = (
    pd.DataFrame({
    'column': df_fraud_aggregated.columns,
        'num_missing': df_fraud_aggregated.isna().sum().values
    })
    .assign(
        pct_missing=lambda d: d['num_missing'] / total * 100,
        has_missing=lambda d: d['num_missing'] > 0
    )
)
print (report)

                                        column  num_missing  pct_missing   
0                                    client_id            0     0.000000  \
1                                       target            0     0.000000   
2                                      disrict            0     0.000000   
3                                       region            0     0.000000   
4                                  client_catg            0     0.000000   
5                     f_invoive_date_diff_days         4212     3.108648   
6                 f_invoive_date_median_months         4212     3.108648   
7                  f_invoive_date_median_years         4212     3.108648   
8               f_counter_statue_error_occured            0     0.000000   
9                            f_counter_regions            0     0.000000   
10                       f_t_region_fraud_rate            0     0.000000   
11       f_region_median_billing_frequence_per            2     0.001476   
12  f_region

In [6]:
X = df_fraud_aggregated.drop(columns=["client_id", "target"])
#X = df_fraud_aggregated.drop(columns=["client_id", "target", "region", "disrict", "client_catg"])
X.fillna(0, inplace=True)  
y = df_fraud_aggregated["target"]

# 2. Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

# 3. Feature-Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 4. Modell-Definition und Training
model = LogisticRegression(
    class_weight='balanced',  # hier werden die Klassengewichte automatisch angepasst
    solver='liblinear',       # geeignet für kleine bis mittelgroße Datensätze
    random_state=42
)
model.fit(X_train_scaled, y_train)

# 5. Evaluation
y_pred = model.predict(X_test_scaled)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[25817 12561]
 [  863  1407]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.67      0.79     38378
         1.0       0.10      0.62      0.17      2270

    accuracy                           0.67     40648
   macro avg       0.53      0.65      0.48     40648
weighted avg       0.92      0.67      0.76     40648

